In [1]:
path = "mnt/blobmount/"

In [2]:
# IMPORTS
import matplotlib.pyplot as plt
import numpy as np
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.regression import LinearRegression, LinearRegressionModel, RandomForestRegressor, RandomForestRegressionModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.linalg import SparseVector

In [3]:
def parse_csv(s):
  return list(map(float, s.split(',')))

def filter_outliers(vals):
  # label
  if not (-7.0 <= vals[0] and vals[0] <= 103.0):
    return False
  # p_score
  if not (-7.0 <= vals[-2] and vals[-2] <= 103.0):
    return False
  
  return True

def normalize(vals):
  # num_words
  vals[-1] = (vals[-1] - 17.13712707395722)/32.30891863874633
  # p_score
  vals[-2] = (vals[-2] - 5.390716682669022)/13.664352804935564
  # p_time
  vals[-4] = (vals[-4] - 9761.223722159446)/50008.29178605555
  return vals
  
def build_tup(vals):
  label = vals[0]
  subreddit_id = int(vals[1])
  
  index = 0
  feat_vec = []
  
  feat_vec.append((subreddit_id, 1))
  index += 100
  for val in vals[2:]:
    if val:
      feat_vec.append((index, val))
    index += 1
    
  return (label, SparseVector(index, feat_vec))
  
def read_csv(filepath):
  return (sc.textFile(filepath, minPartitions=96)
          .map(parse_csv)
          .filter(filter_outliers)
          .map(normalize)
          .map(build_tup)
          .toDF(['label', 'features']))

In [4]:
train_df = read_csv(path + "full_processed_train.csv").cache()
val_df = read_csv(path + "full_processed_val.csv").cache()
test_df = read_csv(path + "full_processed_test.csv").cache()

In [5]:
regEval = RegressionEvaluator(metricName="rmse")

def print_evaluation(df):
  # RMSE metric
  rmse = regEval.evaluate(df)
  print("RMSE: %.2f" % rmse)

  # Mean absolute error
  mae = regEval.evaluate(df, {regEval.metricName: "mae"})
  print("MAE: {0:.2f}".format(mae))
  
  # R^2 (coefficient of determination) metric
  r2 = regEval.evaluate(df, {regEval.metricName: "r2"})
  print("R^2: {0:.2f}".format(r2))

### Baseline

In [7]:
train_label_average = train_df.selectExpr("avg(label)").first()['avg(label)']
baseline_predictions_rdd = test_df.rdd.map(lambda row: (train_label_average, row['label']))
baseline_predictions_df = sqlContext.createDataFrame(baseline_predictions_rdd, ["prediction", "label"])

print_evaluation(baseline_predictions_df)

RMSE: 8.67
MAE: 4.35
R^2: -0.00

### Linear Regression

In [9]:
# Learn best model via cross-validation
best_score, best_param, lr_cv_model = float('inf'), None, None

for reg_param in [10**i for i in range(1, -6, -1)] + [0]:
  lr = LinearRegression(maxIter=100, regParam=reg_param, elasticNetParam=1.0)
  model = lr.fit(train_df)
  
  predictions_df = model.transform(val_df)
  score = regEval.evaluate(predictions_df)
  
  if score < best_score:
    best_score = score
    best_param = reg_param
    lr_cv_model = model

lr_cv_model.write().overwrite().save("mnt/blobmount/lr_cv_model") 

In [10]:
lr_cv_model = LinearRegressionModel.load("mnt/blobmount/lr_cv_model")

# Model Evaluation
lr_predictions_df = lr_cv_model.transform(test_df)
print_evaluation(lr_predictions_df)

RMSE: 8.20
MAE: 3.97
R^2: 0.10

### Random Forests

In [12]:
# Learn the best model via cross-validation
best_score, best_trees, best_depth, rf_cv_model = float('inf'), None, None, None

from datetime import datetime

for trees in [10, 20, 30]:
  for depth in [5, 10, 15]:
    rf = RandomForestRegressor(numTrees=trees, maxDepth=depth)
    model = rf.fit(train_df)

    predictions_df = model.transform(val_df)
    score = regEval.evaluate(predictions_df)
    
    if score < best_score:
      best_score = score
      best_trees = trees
      best_depth = depth
      rf_cv_model = model

rf_cv_model.write().overwrite().save("mnt/blobmount/rf_cv_model")

In [13]:
rf_cv_model = RandomForestRegressionModel.load("mnt/blobmount/rf_cv_model")

# Model Evaluation
rf_predictions_df = rf_cv_model.transform(test_df)
print_evaluation(rf_predictions_df)

RMSE: 7.86
MAE: 3.63
R^2: 0.18

### Visualizations / Miscellaneous

In [15]:
def unpack(row):
  return [row['label'], row['prediction'], row['label'] - row['prediction']]

data = np.array(rf_predictions_df.rdd.map(unpack).collect())

In [16]:
# Plot predictions against labels
fig, ax = plt.subplots()
ax.scatter(data[:,0], data[:,1], s=0.1, alpha=0.01)
ax.set_aspect("equal")
ax.set_xlabel("Label")
ax.set_ylabel("Prediction")
ax.set_title("Label vs Prediction")
display(fig)

In [17]:
# Plot Residuals

fig, ax = plt.subplots()
ax.hist(data[:,2], bins=list(range(-20, 41)))
ax.set_xlabel("Residual")
ax.set_ylabel("Frequency")
ax.set_title("Residuals (Label - Prediction)")
display(fig)

In [18]:
# Feature importances
importances = rf_cv_model.featureImportances
pairs = [(i, importances[i]) for i in range(len(importances))]
sorted_pairs = sorted(pairs, key=lambda tup: -tup[1])

for i in range(len(importances)):
  print(sorted_pairs[i])

(155, 0.5478942470329803)
(153, 0.21023239114578351)
(100, 0.048346069617015354)
(139, 0.017113927110949303)
(156, 0.016823912913421977)
(154, 0.016489298731171786)
(134, 0.011891194078339664)
(138, 0.008883057447868648)
(101, 0.008021712044612304)
(119, 0.0077895769222090395)
(143, 0.006076766614343941)
(0, 0.005347172375113375)
(124, 0.004901606303173294)
(133, 0.004805162706749613)
(120, 0.00444735292366482)
(150, 0.0035314780885984122)
(149, 0.0031678368447532555)
(108, 0.0029861978716631096)
(105, 0.002645226535168148)
(123, 0.0023517904392151728)
(127, 0.0023362735427904283)
(129, 0.002190472732915277)
(152, 0.0019471030324398734)
(137, 0.0019271663833207494)
(122, 0.0018908453314301027)
(117, 0.0017552479873118412)
(130, 0.001631220024615947)
(148, 0.0015838051601463545)
(125, 0.0015686247578295769)
(136, 0.001429342721287045)
(128, 0.0014097358735509838)
(106, 0.0013661517108794748)
(42, 0.0013599533352410226)
(118, 0.001331288917616152)
(140, 0.0013227713383556486)
(115, 0.0013021301944423154)
(112, 0.0012575962579222026)
(147, 0.0012513397963996092)
(131, 0.0011963243906156073)
(107, 0.0011899502782381363)
(114, 0.0011406115521500652)
(30, 0.001134633640720935)
(1, 0.001124421211088428)
(132, 0.0011023212265571152)
(3, 0.0011014752062574894)
(109, 0.0010969912744668935)
(104, 0.0010917311718322323)
(55, 0.0010438715092738599)
(146, 0.001003822635779291)
(113, 0.0010037571079577394)
(135, 0.000998079437400666)
(121, 0.0009970059242823885)
(126, 0.0009921780479220678)
(111, 0.0009845507151156263)
(142, 0.0009741659729183624)
(102, 0.000967413281215675)
(110, 0.0009631916882674989)
(103, 0.0009256984307330155)
(144, 0.0009011914990161232)
(145, 0.0008911991109768882)
(141, 0.0008483683959001812)
(68, 0.000775700667547711)
(116, 0.0007739568378852763)
(151, 0.0007231629219229495)
(14, 0.0007118444425527833)
(95, 0.0005709819608252409)
(4, 0.00042111158098481375)
(97, 0.0004194002078369314)
(70, 0.00040712083214909954)
(10, 0.0004006917425809053)
(77, 0.00037687402262918577)
(29, 0.0003654316651353959)
(13, 0.00028587295054910785)
(2, 0.0002681341754034104)
(37, 0.0002540735558580101)
(8, 0.00020301611460636244)
(54, 0.0002014481118032927)
(47, 0.00019635416809752967)
(67, 0.00019145821695542358)
(5, 0.00017636478309485747)
(7, 0.00015997866892169385)
(44, 0.00015585184980988404)
(12, 0.00014229316854799524)
(80, 0.00014041868345699644)
(49, 0.00013728343558119203)
(91, 0.00013562696660532754)
(11, 0.00013539593762937582)
(96, 0.00012742898883056464)
(21, 0.0001252234872774523)
(88, 0.0001211702232706745)
(71, 0.00011999742284012993)
(84, 0.00011664195137234384)
(28, 0.00011515246399487204)
(22, 0.00011313443570960452)
(58, 0.00011303782080427554)
(50, 0.00011284958679696105)
(9, 0.00011048162276102336)
(40, 0.00011046026210689162)
(26, 0.00010868589356800219)
(85, 0.00010758183800957799)
(17, 0.00010633227442606686)
(76, 0.00010568246099301368)
(86, 0.00010477036732624942)
(46, 0.00010448004005230144)
(6, 0.00010147735598152519)
(15, 9.967865048390497e-05)
(66, 9.678390130884087e-05)
(43, 9.480672829450344e-05)
(52, 9.164969380170762e-05)
(45, 9.055270398393954e-05)
(16, 8.815657394680061e-05)
(38, 8.799165361330903e-05)
(69, 8.78622793324685e-05)
(20, 8.567005049190803e-05)
(27, 7.45946977366279e-05)
(78, 7.329466765980802e-05)
(53, 7.170541562799065e-05)
(89, 7.072832680909842e-05)
(32, 7.029548114494251e-05)
(57, 7.02908839910269e-05)
(98, 6.747141302709415e-05)
(63, 6.624577310500041e-05)
(59, 6.518030405571841e-05)
(31, 6.471905450568924e-05)
(34, 6.447830645531954e-05)
(82, 6.378498936427974e-05)
(93, 6.287749837859116e-05)
(25, 6.0071836479237426e-05)
(87, 5.947154160891086e-05)
(62, 5.6753004599808006e-05)
(56, 5.6580040397224144e-05)
(33, 5.5660531362763325e-05)
(75, 5.5533493198059146e-05)
(36, 5.280845813115847e-05)
(94, 5.2721550351290296e-05)
(74, 4.8849167475952675e-05)
(64, 4.860599821667176e-05)
(72, 4.716177091677223e-05)
(41, 4.6557816609432474e-05)
(79, 4.625543277168883e-05)
(90, 4.6255233763221105e-0